<center> 
<h3><b> Adnan Rashid </b><\h3>
<br><br>
    <nav>
    | <a href="00-Contents-Setup.ipynb"> Home Page </a> | 
    <a href="http://bit.ly/cka_notes_original"> Kubernetes Notes </a> |
    <a href="https://adnan.study"> Website </a> |
    <a href="https://www.instagram.com/adnans_techie_studies/"> Instagram </a> |
    <a href="https://www.linkedin.com/in/adnanrashid1/"> LinkedIn </a> |
    </nav>
</center>

<h1 style="background:black"><code style="background:black;color:white"><center>IAM Groups to Manage Kubernetes Cluster Access</center></code></h1>

<div class="alert alert-block alert-warning">
<font style="color:black">
<center> In this section we will learn how to simplify access to different parts of the kubernetes clusters depending on IAM Groups 
    
</div>
</font>
</center>

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/17.jpg" width="40%" height="40%" border=5/>

* [Kubernetes Authentication](#kubeauth) 
* [Create IAM Roles](#iamroles) 
* [Create IAM Groups](#iamgroups) 
* [Create IAM Users](#iamusers) 
* [Configure Kubernetes RBAC](#kuberbac)
* [Configure Kubernetes Role Access](#kuberoleaccess) 
* [Test EKS Access](#testeksaccess)
* [Cleanup](#groupscleanup) 

<br><br>
<blockquote>In the intro we have seen how we can give access to individual users to Kubernetes <br><br>
If you have different teams which needs different kind of cluster access, it would be difficult to manually add or remove access for each EKS cluster you want to give them or remove access from.<br><br>

We can leverage AWS IAM Groups to easily add or remove users and give them permission to whole cluster, or just part of it depending on which groups they belong to. 

## Pre-requisite

* Check your variables are loaded

In [ ]:
%store

* Load in variable store

In [ ]:
%store -r 

* Load variables into notebook

In [ ]:
%env AWS_PROFILE=$AWS_PROFILE
%env AWS_DEFAULT_REGION=$AWS_DEFAULT_REGION
%env ACCOUNT_ID=$ACCOUNT_ID

* Check identity 

In [ ]:
!aws sts get-caller-identity

<br>

---
---

<br>

## <br><br>Creating IAM Roles

Create 3 Roles

* **k8sAdmin** role which will have admin rights in our EKS cluster
* **k8sDev** role which will gives access to developers namespace in our EKS cluster
* **k8sInteg** role which will gives access to integration namespace our EKS cluster

In [ ]:
%%bash

POLICY=$(echo -n '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":"arn:aws:iam::'; echo -n "$ACCOUNT_ID"; echo -n ':root"},"Action":"sts:AssumeRole","Condition":{}}]}')
echo POLICY=$POLICY

aws iam create-role \
  --role-name k8sAdmin \
  --description "Kubernetes administrator role (for AWS IAM Authenticator for Kubernetes)." \
  --assume-role-policy-document "$POLICY" \
  --output text \
  --query 'Role.Arn'

aws iam create-role \
  --role-name k8sDev \
  --description "Kubernetes developer role (for AWS IAM Authenticator for Kubernetes)." \
  --assume-role-policy-document "$POLICY" \
  --output text \
  --query 'Role.Arn'
  
aws iam create-role \
  --role-name k8sInteg \
  --description "Kubernetes role for integration namespace in quick cluster." \
  --assume-role-policy-document "$POLICY" \
  --output text \
  --query 'Role.Arn'

<br><br>
<blockquote>
In this example, the assume-role-policy allows the root account to assume the role. 

* We are going to allow specific groups to also be able to assume those roles 

Because the above roles are only used to authenticate within the k8s cluster, they don't necessarily need to have AWS permissions 

* We will only use them to allow some IAM groups to assume this role in order to have access to EKS kubernetes cluster. 

## <br><br>Create IAM Groups

<blockquote>We want to have different IAM users which will be added to specific IAM groups in order to have different rights in the kubernetes cluster 

* We will define 3 groups 
    * **k8sAdmin** - users from this group will have admin rights on the kubernetes cluster
    * **k8sDev** - users from this group will have full access only in the development namespace of the cluster
    * **k8sInteg** - users from this group will have access to integration namespace.<br>
    

The users from **k8sDev** and **k8sInteg** groups will only have access to namespaces where we will define kubernetes RBAC access for their associated kubernetes role. 

We'll see this but first, let's create the groups. 

#### <br><br>Create k8sAdmin IAM Group

* The <b>k8sAdmin</b> group will be allowed to assume the <b>k8sAdmin</b> IAM Role 

In [ ]:
!aws iam create-group --group-name k8sAdmin

* Lets add a policy on our group which will allow users from this group to assume our <b>k8sAdmin</b> role 

In [ ]:
%%bash 

ADMIN_GROUP_POLICY=$(echo -n '{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowAssumeOrganizationAccountRole",
      "Effect": "Allow",
      "Action": "sts:AssumeRole",
      "Resource": "arn:aws:iam::'; echo -n "$ACCOUNT_ID"; echo -n ':role/k8sAdmin"
    }
  ]
}')
echo ADMIN_GROUP_POLICY=$ADMIN_GROUP_POLICY

aws iam put-group-policy \
--group-name k8sAdmin \
--policy-name k8sAdmin-policy \
--policy-document "$ADMIN_GROUP_POLICY"

#### <br><br>Create k8sDev IAM Group

* The <b>k8sDev</b> Group will be allowed to assume the <b>k8sDev</b> IAM Role

In [ ]:
!aws iam create-group --group-name k8sDev

* Let's add a Policy on our group which will allow users from this group to assume our <b>k8sDev</b> Role 

In [ ]:
%%bash 

DEV_GROUP_POLICY=$(echo -n '{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowAssumeOrganizationAccountRole",
      "Effect": "Allow",
      "Action": "sts:AssumeRole",
      "Resource": "arn:aws:iam::'; echo -n "$ACCOUNT_ID"; echo -n ':role/k8sDev"
    }
  ]
}')
echo DEV_GROUP_POLICY=$DEV_GROUP_POLICY

aws iam put-group-policy \
--group-name k8sDev \
--policy-name k8sDev-policy \
--policy-document "$DEV_GROUP_POLICY"

#### <br><br>Create k8sInteg IAM Group

In [ ]:
!aws iam create-group --group-name k8sInteg

* Lets add a Policy on our group which will allow users from this group to assume our <b>k8sInteg</b> role

In [ ]:
%%bash 

INTEG_GROUP_POLICY=$(echo -n '{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowAssumeOrganizationAccountRole",
      "Effect": "Allow",
      "Action": "sts:AssumeRole",
      "Resource": "arn:aws:iam::'; echo -n "$ACCOUNT_ID"; echo -n ':role/k8sInteg"
    }
  ]
}')
echo INTEG_GROUP_POLICY=$INTEG_GROUP_POLICY

aws iam put-group-policy \
--group-name k8sInteg \
--policy-name k8sInteg-policy \
--policy-document "$INTEG_GROUP_POLICY"

* We should have 3 groups now, lets check

In [ ]:
!aws iam list-groups

## <br><br>Create IAM Users 

* In order to test our scenarios, we will create 3 users, one for each groups we created 

In [ ]:
!aws iam create-user --user-name PaulAdmin
!aws iam create-user --user-name JeanDev
!aws iam create-user --user-name PierreInteg

* Add users to associated groups

In [ ]:
!aws iam add-user-to-group --group-name k8sAdmin --user-name PaulAdmin
!aws iam add-user-to-group --group-name k8sDev --user-name JeanDev
!aws iam add-user-to-group --group-name k8sInteg --user-name PierreInteg

* Check users are correctly added in their groups

In [ ]:
!aws iam get-group --group-name k8sAdmin
!aws iam get-group --group-name k8sDev
!aws iam get-group --group-name k8sInteg

* Create Some access keys so we can toggle between the users <br>

<div class="alert alert-block alert-info">
<font style="color:black">
<center>Saving in the temp directory belows makes it easier to clean up at the end

In [ ]:
!aws iam create-access-key --user-name PaulAdmin | tee temp/PaulAdmin.json
!aws iam create-access-key --user-name JeanDev | tee temp/JeanDev.json
!aws iam create-access-key --user-name PierreInteg | tee temp/PierreInteg.json

<br><br>So far we have done the following 

* <b> PaulAdmin </b> is in the <b> k8sAdmin </b> group and will be able to assume the <b> k8sAdmin </b> role 
* <b> JeanDev </b> is in <b> k8sDev </b> Group and will be able to assume IAM role <b> k8sDev </b>
* <b> PierreInteg </b> is in <b> k8sInteg </b> group and will be able to assume IAM role <b> k8sInteg </b> 

## <br><br>Configure Kubernetes RBAC

#### Create Kubernetes namespaces 

* <b> development </b> namespace will be accessible for IAM users from <b> k8sDev </b> group 
* <b> integration </b> namespace will be accessible for IAM users from <b> k8sInteg </b> group 

In [ ]:
!kubectl create namespace integration 
!kubectl create namespace development 

#### <br><br>Configuring access to development namespace

* We create a kubernetes <i> role </i> and <i> rolebinding </i> in the development namespace giving full access to the kubernetes user <b> dev-user </b> 

In [ ]:
%%bash

cat << EOF | kubectl apply -f - -n development
kind: Role
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: dev-role
rules:
  - apiGroups:
      - ""
      - "apps"
      - "batch"
      - "extensions"
    resources:
      - "configmaps"
      - "cronjobs"
      - "deployments"
      - "events"
      - "ingresses"
      - "jobs"
      - "pods"
      - "pods/attach"
      - "pods/exec"
      - "pods/log"
      - "pods/portforward"
      - "secrets"
      - "services"
    verbs:
      - "create"
      - "delete"
      - "describe"
      - "get"
      - "list"
      - "patch"
      - "update"
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: dev-role-binding
subjects:
- kind: User
  name: dev-user
roleRef:
  kind: Role
  name: dev-role
  apiGroup: rbac.authorization.k8s.io
EOF

<br><br>The role we define will give full access to everything in that namespace. 

It is a Role, and not a ClusterRole, so it is going ot be applied only in the <b> development </b> namespace 

#### <br><br>Configuring Access to Integration Namespace

* We create a kubernetes <i> role</i> and <i> rolebinding </i> in the integration namespace for full access with the kubernetes user <b> integ-user</b> 

In [ ]:
%%bash

cat << EOF | kubectl apply -f - -n integration
kind: Role
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: integ-role
rules:
  - apiGroups:
      - ""
      - "apps"
      - "batch"
      - "extensions"
    resources:
      - "configmaps"
      - "cronjobs"
      - "deployments"
      - "events"
      - "ingresses"
      - "jobs"
      - "pods"
      - "pods/attach"
      - "pods/exec"
      - "pods/log"
      - "pods/portforward"
      - "secrets"
      - "services"
    verbs:
      - "create"
      - "delete"
      - "describe"
      - "get"
      - "list"
      - "patch"
      - "update"
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: integ-role-binding
subjects:
- kind: User
  name: integ-user
roleRef:
  kind: Role
  name: integ-role
  apiGroup: rbac.authorization.k8s.io
EOF

<br><br>The role we define will give full access to everything in that namespace. 

It is a <i> Role</i> and not a <i> ClusterRole </i>, so it is going to be applied only in the <b> integration </b> namespace 

## <br><br>Configure Kubernetes Role Access 

#### Give Access to our IAM Roles to EKS Cluster 

<br>In order to give access to the IAM Roles we defined previously to our EKS cluster, we need to add specific <b> mapRoles </b> to the <i> aws-auth</i> ConfigMap

The Advantage of using Role to access the cluster instead of specifying directly IAM users is that it will be easier to manage 
* We dont have to update the ConfigMap each time we want to add or remove users 
* We will just need to add or remove users from the IAM Group and we just configure the ConfigMap to allow the IAM Role associated to the IAM Group

#### <br><br>Update the aws-auth ConfigMap to allow our IAM roles

<br>The <b> aws-auth </b> ConfigMap from the kube-system namespace must be edited in order to allow new arn Groups. 

This file makes the mapping between IAM role and k8S RBAC rights. 

* We can edit it using eksctl 

In [ ]:
!eksctl create iamidentitymapping --cluster eksworkshop-eksctl --arn arn:aws:iam::$ACCOUNT_ID:role/k8sDev --username dev-user 
!eksctl create iamidentitymapping --cluster eksworkshop-eksctl --arn arn:aws:iam::$ACCOUNT_ID:role/k8sInteg --username integ-user 
!eksctl create iamidentitymapping --cluster eksworkshop-eksctl --arn arn:aws:iam::$ACCOUNT_ID:role/k8sAdmin --username admin --group system:masters

* Check the config map

In [ ]:
!kubectl get cm -n kube-system aws-auth -o yaml

* We can leverage eksctl to get a list of all identity managed in our cluster 

In [ ]:
!eksctl get iamidentitymapping --cluster eksworkshop-eksctl

<br><br>Here we have created:

* a RBAC role for K8sAdmin, that we map to admin user and give access to <b>system:masters</b> kubernetes Groups (so that it has Full Admin rights)
* a RBAC role for k8sDev that we map on dev-user in development namespace
* a RBAC role for k8sInteg that we map on integ-user in integration namespace

We will see on next section how we can test it.

## <br><br> Test EKS Access

#### Automate assumerole with awscli 

It is possible to automate the retrival of temporary credentials for the assumed role by configuring the awscli using .aws/config and .aws/credentials

<b>Note:</b> The following few commands will wipe your default config, so back it up first 

* add in ~/.aws/config

In [ ]:
%%bash

if [ ! -d ~/.aws ]; then
  mkdir ~/.aws
fi

cat << EoF >> ~/.aws/config
[profile admin]
role_arn=arn:aws:iam::${ACCOUNT_ID}:role/k8sAdmin
source_profile=eksAdmin

[profile dev]
role_arn=arn:aws:iam::${ACCOUNT_ID}:role/k8sDev
source_profile=eksDev

[profile integ]
role_arn=arn:aws:iam::${ACCOUNT_ID}:role/k8sInteg
source_profile=eksInteg

EoF

#### create ~/.aws/credentials

* Use the credentials we generated previously here

In [ ]:
%%bash 

cat << EoF > ~/.aws/credentials


[eksAdmin]
aws_access_key_id=$(jq -r .AccessKey.AccessKeyId temp/PaulAdmin.json)
aws_secret_access_key=$(jq -r .AccessKey.SecretAccessKey temp/PaulAdmin.json)


[eksDev]
aws_access_key_id=$(jq -r .AccessKey.AccessKeyId temp/JeanDev.json)
aws_secret_access_key=$(jq -r .AccessKey.SecretAccessKey temp/JeanDev.json)


[eksInteg]
aws_access_key_id=$(jq -r .AccessKey.AccessKeyId temp/PierreInteg.json)
aws_secret_access_key=$(jq -r .AccessKey.SecretAccessKey temp/PierreInteg.json)

EoF

#### <br><br> Test this with dev/admin/integ profile

* We should see k8sDev in the Arn

In [ ]:
!aws sts get-caller-identity --profile dev

In [ ]:
!aws sts get-caller-identity --profile admin

In [ ]:
!aws sts get-caller-identity --profile integ

### <br><br> Using AWS profiles with Kubectl config file

* It is also possible to specify the AWS_PROFILE to use with the aws-iam-authenticator in the .kube/config file, so that it will use the appropriate profile

#### <br><br>Dev Profile 

* Edit the file in /temp/kubeconfig-dev such that the username is all 'dev' as per the below

You can just do a 'find' and 'replace' 

<br>We added the **--profile dev** parameter to our kubectl config file, so that this will ask kubectl to use our IAM role associated to our dev profile, and we rename this context using suffix **-dev**

With this configuration we should be able to interract with the <b> development </b> namespace, because it is as our RBAC role defined

In [ ]:
%env AWS_PROFILE=dev
%env KUBECONFIG=temp/kubeconfig-dev

In [ ]:
!eksctl utils write-kubeconfig eksworkshop-eksctl

* Check your identity

In [ ]:
!aws sts get-caller-identity

* Lets create a pod 

In [ ]:
!kubectl run --generator=run-pod/v1 nginx-dev --image=nginx -n development

* List the pods

In [ ]:
!kubectl get pods -n development

<br><br>However we cannot list pods in the other namespace (Integration)

* If you find that this works, it could be that you are still not assuming the correct user and instead your original administrator one.

In [ ]:
!kubectl get pods -n integration

<br>We should get <b> error </b>

#### <br><br>Test with Integ Profile 

* Edit the temp/kubeconfig-dev file to add the following content and set the current context to be integ

* Overall the context section should look like this

* Then to the temp/kubeconfig-dev file add integ to the users block 

* Set your AWS Profile to be the integ

In [ ]:
%env AWS_PROFILE=integ

In [ ]:
!kubectl run --generator=run-pod/v1 nginx-integ --image=nginx -n integration

* We can list the pods

In [ ]:
!kubectl get pods -n integration

* However this time the development namespace is not accessible

In [ ]:
!kubectl get pods -n development

<br> We expect an <b>error</b> here 

#### <br><br>Test with admin profile

* Edit the temp/kubeconfig-dev file to add the following content and set the current context to be admin

* Overall the context section should look like this

* Then to the temp/kubeconfig-dev file add integ to the users block 

* Set AWS Profile to be admin 

In [ ]:
%env AWS_PROFILE=admin

* lets create a pod in the default namespace 

In [ ]:
!kubectl run --generator=run-pod/v1 nginx-admin --image=nginx

* we can list ALL pods in the namespace

In [ ]:
!kubectl get pods -A

## <br><br> Switching between different contexts

It is possible to merge several kubernetes API access in the same KUBECONFIG file, or just tell kubectl several file to lookup at once

In the above we used the same file but it could have been seperate files and we set that each time 

* Kubectx can quiickly list and switch context run the following on a terminal 

<p style="background:black">
<code style="background:black;color:white">> curl -sSLO https://raw.githubusercontent.com/ahmetb/kubectx/master/kubectx && chmod 755 kubectx && sudo mv kubectx /usr/local/bin
</code>
</p>

## <br><br> Conclusion 

<blockquote> 

    In this module we configured EKS to provide finer access to users combining IAM Groups and Kubernetes RBAC 
    
    You are able to create different groups depending on needs
    
    Configure their associated RBAC access in your cluster and simply add or remove users from the group to remove them access to your cluster 

    Users will only have to configure their awscli in order to automatically retrive their associated rights in your cluster
    
</blockquote>

## <br><br> Cleanup

* Set AWS Profile by to main admin

In [ ]:
%env AWS_PROFILE=$AWS_PROFILE
%env KUBECONFIG=

In [ ]:
!kubectl delete namespace development integration
!kubectl delete pod nginx-admin

In [ ]:
!eksctl delete iamidentitymapping --cluster eksworkshop-eksctl --arn arn:aws:iam::$ACCOUNT_ID:role/k8sAdmin
!eksctl delete iamidentitymapping --cluster eksworkshop-eksctl --arn arn:aws:iam::$ACCOUNT_ID:role/k8sDev
!eksctl delete iamidentitymapping --cluster eksworkshop-eksctl --arn arn:aws:iam::$ACCOUNT_ID:role/k8sInteg

In [ ]:
!aws iam remove-user-from-group --group-name k8sAdmin --user-name PaulAdmin
!aws iam remove-user-from-group --group-name k8sDev --user-name JeanDev
!aws iam remove-user-from-group --group-name k8sInteg --user-name PierreInteg

In [ ]:
!aws iam delete-group-policy --group-name k8sAdmin --policy-name k8sAdmin-policy 
!aws iam delete-group-policy --group-name k8sDev --policy-name k8sDev-policy 
!aws iam delete-group-policy --group-name k8sInteg --policy-name k8sInteg-policy 

In [ ]:
!aws iam delete-group --group-name k8sAdmin
!aws iam delete-group --group-name k8sDev
!aws iam delete-group --group-name k8sInteg

In [ ]:
!aws iam delete-access-key --user-name JeanDev --access-key-id=$(jq -r .AccessKey.AccessKeyId temp/JeanDev.json)
!aws iam delete-access-key --user-name PaulAdmin --access-key-id=$(jq -r .AccessKey.AccessKeyId temp/PaulAdmin.json)
!aws iam delete-access-key --user-name PierreInteg --access-key-id=$(jq -r .AccessKey.AccessKeyId temp/PierreInteg.json)

In [ ]:
!aws iam delete-user --user-name PaulAdmin
!aws iam delete-user --user-name JeanDev
!aws iam delete-user --user-name PierreInteg

In [ ]:
!aws iam delete-role --role-name k8sAdmin
!aws iam delete-role --role-name k8sDev
!aws iam delete-role --role-name k8sInteg

* Remove credentials and kubeconfig and all credentials saved 

In [ ]:
!rm temp/JeanDev.json
!rm temp/PaulAdmin.json
!rm temp/PierreInteg.json

* Reset aws credentials and config file

In [ ]:
!rm ~/.aws/{config,credentials}
!aws configure set default.region ${AWS_REGION}

In [ ]:
!cat ~/.aws/config
!cat ~/.aws/credential

* Set kubeconfig back to original location

In [ ]:
!aws eks update-kubeconfig --name eksworkshop-eksctl --region $AWS_DEFAULT_REGION --profile $AWS_PROFILE

* Delete nginx-admin

In [113]:
!kubectl delete pod nginx-admin

pod "nginx-admin" deleted


<br><br>
<center>
<h1 style="background:black">
<code style="background:black;color:white">
IAM Roles for Service Accounts
</code>

<div class="alert alert-block alert-warning">
<font style="color:black">
<center> In this section we will create an OIDC identity provider, an IAM Role for Service account and deploy a sample pod 

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/18.jpg" width="40%" height="40%" border=5/>

## Fine-Grained IAM Roles for Service Accounts

* In Kubernetes version 1.12, support was added for a new <b> ProjectedServiceAccountToken </b> feature, which is an OIDC JSON web token that also contains the service account identity, and supports a configurable audience. 
* Amazon EKS now hosts a public OIDC discovery endpoint per cluster containing the signing keys for the ProjectedServiceAccountToken JSON web tokens so external systems, like IAM, can validate and accept the Kubernetes-issued OIDC tokens. 
* OIDC federation access allows you to assume IAM roles via the Secure Tokens Service (STS), enabling authentication with OIDC provider, receiving a JSON Web Token (JWT), which in turn can be used to assume an IAM role. 
* Kubernetes, on the other hand, can issue so-called projected service account tokens, which happen to be valid OIDC JWTs for pods. 

* Our setup equips each pod with a cryptographically-signed token that can be verified by STS against the OIDC provider of your choice to establish the pod's identity. 

new credential provider <b>"sts:AssumeRoleWithWebIdentity"</b>

## <br><br>Preparation

#### <br><br> Enabling IAM Roles for Service Accounts on your Cluster

* The IAM roles for service accounts feature is available on new Amazon EKS Kubernetes version 1.14 clusters and clusters that were updated to versions 1.14 or 1.13 on or after September 3rd, 2019. 


<table>
<thead>
<tr><th>1.14</th><th>1.13</th></tr>
</thead>
<tbody>
<tr><td>Client Version:v1.14.6-eks-5047ed</td><td>v1.13.7</td>
<tr><td>Server Version: v1.14.6-eks-5047ed</td><td>v1.13.10-eks-5ac0f1</td>
</tbody>
</table>

You must use at least version 1.18.15 of the AWS CLI to recieve the proper output from this command

In [ ]:
!aws --version

* Retrieve OpenID Connect issuer URL

In [ ]:
!aws eks describe-cluster --name eksworkshop-eksctl --query cluster.identity.oidc.issuer --output text

## <br><br> Create an OIDC Identity Provider

#### <br><br> To create an IAM OIDC identity provider for your cluster with eksctl 

To use IAM roles for service accounts in your cluster, you must create an OIDC identity provider in the IAM console

* Check eksctl is at least version 0.5.1

In [ ]:
!eksctl version

 * Create your OIDC identity provider for your cluster 

In [ ]:
!eksctl utils associate-iam-oidc-provider --cluster eksworkshop-eksctl --approve 

<br><Br>If you go to IAM Console, you will see OIDC provider has created for your cluster. 

## <br><br> Creating an IAM Role for Service Account 

#### <br><br> To create an IAM role for your service accounts with eksctl 

You must create an IAM policy that specifies the permissions that you would like the containers in your pod to have 

In this workshop we will use AWS managed policy named <b> 'AmazonS3ReadOnlyAccess' </b> which allow get and list for all S3 resources. 

You must also create a role for your service accounts to use before you associate it with a service account. 

Then you can then attach a specific IAM policy to the role that gives the containers in your pods the permissions you desire.

* Get ARN for AmazonS3ReadOnlyAccess

In [ ]:
!aws iam list-policies --query 'Policies[?PolicyName==`AmazonS3ReadOnlyAccess`].Arn'

* Create an IAM role for your service accounts

In [ ]:
!eksctl create iamserviceaccount --name iam-test --namespace default --cluster eksworkshop-eksctl --attach-policy-arn arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess --approve --override-existing-serviceaccounts

<br><br>The following deploys out a stack called **'eksctl-eksworkshop-eksctl-addon-iamserviceaccount-default-iam-test'** and has created a role for your service accont

## <br><br> Specifying an IAM Role for Service Account

#### <br><br> Specifying an IAM role for your Service Account

In the previous step, we created the IAM role that associated with a service account named <b> iam-test </b> in the cluster and this has already been done for you with the service account you specified when creating the role 

* Be sure your service account iam-test exists

In [ ]:
!kubectl get sa

In [ ]:
!kubectl describe sa iam-test 

## <br><br> Deploy Sample Pod  

Now that we have completed all the necessary configuration to run a Pod with IAM role 
* We will deploy sample Pod to the cluster, and run a test command to see whether it works correctly or not. 

In [ ]:
!kubectl apply -f oidc/iam-pod.yaml

* Make sure your pod is in Running status 

In [ ]:
!kubectl get pod

* Get into the pod (run these in terminal) 

<p style="background:black">
<code style="background:black;color:white">> kubectl exec -it eks-iam-test-5f88448558-fnvgl /bin/bash
</code>
</p>

* Call sts:AssumeRoleWithWebIdentity, and you will see AccessKeyId and SecretAccessKey information if configuration is set appropriately 


<p style="background:black">
<code style="background:black;color:white">> aws sts assume-role-with-web-identity \
--role-arn $AWS_ROLE_ARN \
--role-session-name mh9test \
--web-identity-token file://$AWS_WEB_IDENTITY_TOKEN_FILE \
--duration-seconds 1000

* Run awscli to see if it retrieves list of S3 buckets 

<p style="background:black">
<code style="background:black;color:white">> aws s3 ls 

* Run awscli to see if it retrieves list of Amazon Ec2 instances which does not have privileged in the allocated IAM policy 

<p style="background:black">
<code style="background:black;color:white">> aws ec2 describe-instances --region us-west-2 

* We should get an error message 

<p style="background:black">
<code style="background:black;color:white">> An error occurred (UnauthorizedOperation) when calling the DescribeInstances operation: You are not authorized to perform this operation.
</code>
</p>

## <br><br> Cleanup 

* Remove sample application 

In [ ]:
!kubectl delete -f oidc/iam-pod.yaml 

* Remove IAM role and Service Account stack from cloudformation

In [ ]:
!eksctl delete iamserviceaccount --name iam-test --namespace default --cluster eksworkshop-eksctl

* Remove OIDC Integration

In [ ]:
!aws eks describe-cluster --name eksworkshop-eksctl --output json | jq -r .cluster.identity.oidc.issuer | sed -e "s*https://**"

In [ ]:
!aws iam delete-open-id-connect-provider --open-id-connect-provider-arn arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.eu-west-1.amazonaws.com/id/272A2E290EB7870EE3BF406E63F6507E